In [ ]:
# This notebook is intended to demonstrate basic use of pyHARM.  It is a work in progress, additions or issues welcome
# Note this requires pyHARM to be installed, unless you move it to the root of this repository
import pyHARM

import numpy as np
import matplotlib.pyplot as plt

# Suppress math warnings. These are just for /0 or sqrt(-)
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
# pyHARM returns IharmDump objects by default, which include not only HARM's primitive variables,
# but the covariant and contravariant 4-vectors of velocity (ucon, ucov) and magnetic field (bcon, bcov)
# (see Gammie '03 for definition of the latter)
# It also includes the instantaneous 4-current jcon at the time of the dump, if available.
# Disable adding the 4-vectors with add_derived=False
# Disable adding the 4-current jcon with add_jcon=False
# You can additionally add conserved variables and zone failures (if these are present in the file) with add_cons, add_fail
dump = pyHARM.load_dump("/data/bh-bd3/eht/GRMHD/MAD/a+0.94/384x192x192_IHARM/dumps/dump_00002000.h5")

In [ ]:
# See all the variables you can request like this in pyHARM/ana/variables.py
# Example of calculating dimensionless magnetization
# Note you can pass just the name of a variable directly,
# or retrieve the variable, do some operations, and pass the resulting ndarray
0.5 * pyHARM.shell_sum(dump, np.fabs(dump['B1']), at_zone=5) / np.sqrt(-pyHARM.shell_sum(dump, 'FM', at_zone=5))

In [ ]:
# Dimensionless magnetization as a function of r
# Note that in most cases you woulnd't probably use 'pretty' for a variable you know.
# It's mostly useful when plotting a lot of different variables with the same code
plt.plot(dump['r'][:,0,0], 0.5 * pyHARM.shell_sum(dump, np.fabs(dump['B1'])) / np.sqrt(- pyHARM.shell_sum(dump, 'FM')))
plt.xlim(0,50)
plt.ylim(0,50)
plt.xlabel(pyHARM.pretty('r'))
plt.ylabel(pyHARM.pretty('phi_b'))
plt.title("Spherically summed radial field by radius")
True

In [ ]:
# Shell-average azimuthal velocity as a function of r is more stable
# Note this is just for a particular dump, too
plt.plot(dump['r'][:,0,0], pyHARM.shell_av(dump, 'u^phi'))
plt.xlim(0,400)
plt.yscale('log')
plt.xlabel(pyHARM.pretty('r'))
plt.ylabel(pyHARM.pretty('u^phi'))
plt.title(r"Shell-average u^{\phi}")
True

In [ ]:
# TODO there's lots of other neat, simple examples...